In [ ]:
import os
import subprocess

LIB = '/home/xgao/anaconda3/lib/python3.7/site-packages/torch/lib/'
FILE = 'libtorch_cuda.so'

def demangle(symbol):
    out = subprocess.Popen(['c++filt', symbol],
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    return stdout.decode().strip()

def get_symbols():
    out = subprocess.Popen(['cuobjdump', '-symbols', os.path.join(LIB, FILE)],
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    l = stdout.decode().split('\n')
    len_ = len('STT_FUNC         STB_GLOBAL STO_ENTRY      ')
    l = [x.strip()[len_:] for x in l if 'modern' in x]
#     l = [x for x in l if 'lambda(float, float)' in demangle(x)]
    return l

def get_asm(symbol):
    out = subprocess.Popen(['cuobjdump', '-sass', '--function', symbol, os.path.join(LIB, FILE)],
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    l = stdout.decode().split('\n')
    l = [x.strip() for x in l if x.count('/*') > 1]
    return '\n'.join(l)

def run():
    import torch
    print(torch.__version__)
    print(torch.version.git_version)
    print()
    for s in get_symbols():
        print('**Symbol:**')
        print(demangle(s))
        print()
        print("**ASM:**")
        print(get_asm(s))
        print()

run()

1.4.0a0+57b180f
57b180f9c5b99381879437ee9db0f0978743282c

**Symbol:**
_ZN2at6native6modern18elementwise_kernelILi512ELi1EZZZNS0_69_GLOBAL__N__45_tmpxft_000030a3_00000000_6_Activation_cpp1_ii_150aa41226leaky_relu_backward_kernelERNS_14TensorIteratorEN3c106ScalarEENKUlvE_clEvENKUlvE1_clEvEUlNS6_4HalfESA_E_Li0EEEviT1_NS_6detail5ArrayIPcXplsr15function_traitsISC_EE5arityLi1EEEE

**ASM:**
/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;               /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                         /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_CTAID.X ;                                  /* 0x0000000000007919 */
/*0030*/                   MOV R5, 0x2 ;                                         /* 0x0000000200057802 */
/*0040*/                   S2R R3, SR_TID.X ;                                    /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R0, 0x200, R3 ;                              /*

/*0000*/                   MOV R1, c[0x0][0x28] ;                 /* 0x00000a0000017a02 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;          /* 0x000000fffffff389 */
/*0020*/                   S2R R4, SR_TID.X ;                     /* 0x0000000000047919 */
/*0030*/                   MOV R5, 0x4 ;                          /* 0x0000000400057802 */
/*0040*/                   S2R R3, SR_CTAID.X ;                   /* 0x0000000000037919 */
/*0050*/                   LEA R4, R3, R4, 0x9 ;                  /* 0x0000000403047211 */
/*0060*/                   IMAD.WIDE R2, R4, R5, c[0x0][0x188] ;  /* 0x0000620004027625 */
/*0070*/                   LDG.E.SYS R7, [R2] ;                   /* 0x0000000002077381 */
/*0080*/                   IMAD.WIDE R4, R4, R5, c[0x0][0x180] ;  /* 0x0000600004047625 */
/*0090*/                   FSETP.GT.AND P0, PT, R7, RZ, PT ;      /* 0x000000ff0700720b */
/*00a0*/              @!P0 FMUL R7, R7, c[0x0][0x168] ;           /* 0x00005a0007078a20 */

/*0000*/                   MOV R1, c[0x0][0x28] ;                                       /* 0x00000a0000017a02 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                                /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_CTAID.X ;                                         /* 0x0000000000007919 */
/*0030*/                   MOV R11, 0x4 ;                                               /* 0x00000004000b7802 */
/*0040*/                   S2R R3, SR_TID.X ;                                           /* 0x0000000000037919 */
/*0050*/                   LEA R0, R0, R3, 0x9 ;                                        /* 0x0000000300007211 */
/*0060*/                   IMAD.WIDE R12, R0, R11, c[0x0][0x180] ;                      /* 0x00006000000c7625 */
/*0070*/                   LDG.E.SYS R3, [R12] ;                                        /* 0x000000000c037381 */
/*0080*/                   IMAD.WIDE R10, R0, R11, c[0x0][0x178] ;                      /* 0x000

/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;                   /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                             /* 0x000000fffffff389 */
/*0020*/                   S2R R15, SR_CTAID.X ;                                     /* 0x00000000000f7919 */
/*0030*/                   IMAD.MOV.U32 R0, RZ, RZ, 0x8 ;                            /* 0x00000008ff007424 */
/*0040*/                   S2R R2, SR_TID.X ;                                        /* 0x0000000000027919 */
/*0050*/                   IMAD R15, R15, 0x200, R2 ;                                /* 0x000002000f0f7824 */
/*0060*/                   IMAD.WIDE R2, R15, R0, c[0x0][0x180] ;                    /* 0x000060000f027625 */
/*0070*/                   LDG.E.64.SYS R2, [R2] ;                                   /* 0x0000000002027381 */
/*0080*/                   IMAD.WIDE R4, R15, R0, c[0x0][0x178] ;                    /* 0x00005e000f047625 */
/*0090*/  

/*0000*/                   MOV R1, c[0x0][0x28] ;                                       /* 0x00000a0000017a02 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                                /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_CTAID.X ;                                         /* 0x0000000000007919 */
/*0030*/                   MOV R11, 0x2 ;                                               /* 0x00000002000b7802 */
/*0040*/                   S2R R3, SR_TID.X ;                                           /* 0x0000000000037919 */
/*0050*/                   LEA R0, R0, R3, 0x9 ;                                        /* 0x0000000300007211 */
/*0060*/                   IMAD.WIDE R10, R0, R11, c[0x0][0x178] ;                      /* 0x00005e00000a7625 */
/*0070*/                   LDG.E.U16.SYS R2, [R10] ;                                    /* 0x000000000a027381 */
/*0080*/                   MOV R14, 0x3a69a091 ;                                        /* 0x3a6

...

/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;         /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                   /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_TID.X ;                              /* 0x0000000000007919 */
/*0030*/                   MOV R5, 0x2 ;                                   /* 0x0000000200057802 */
/*0040*/                   S2R R3, SR_CTAID.X ;                            /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R3, 0x200, R0 ;                        /* 0x0000020003007824 */
/*0060*/                   IMAD.WIDE R2, R0, R5, c[0x0][0x178] ;           /* 0x00005e0000027625 */
/*0070*/                   IMAD.WIDE R4, R0, R5, c[0x0][0x180] ;           /* 0x0000600000047625 */
/*0080*/                   LDG.E.U16.SYS R2, [R2] ;                        /* 0x0000000002027381 */
/*0090*/                   LDG.E.U16.SYS R4, [R4] ;                        /* 0x0000000004047381 */


/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;         /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                   /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_TID.X ;                              /* 0x0000000000007919 */
/*0030*/                   IMAD.MOV.U32 R5, RZ, RZ, 0x2 ;                  /* 0x00000002ff057424 */
/*0040*/                   S2R R3, SR_CTAID.X ;                            /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R3, 0x200, R0 ;                        /* 0x0000020003007824 */
/*0060*/                   IMAD.WIDE R2, R0, R5, c[0x0][0x188] ;           /* 0x0000620000027625 */
/*0070*/                   LDG.E.U16.SYS R2, [R2] ;                        /* 0x0000000002027381 */
/*0080*/                   LDC.U16 R4, c[0x0][0x16a] ;                     /* 0x00005a80ff047b82 */
/*0090*/                   PRMT R4, R4, 0x9910, RZ ;                       /* 0x0000991004047816 */


/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;              /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                        /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_TID.X ;                                   /* 0x0000000000007919 */
/*0030*/                   IMAD.MOV.U32 R5, RZ, RZ, 0x8 ;                       /* 0x00000008ff057424 */
/*0040*/                   S2R R3, SR_CTAID.X ;                                 /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R3, 0x200, R0 ;                             /* 0x0000020003007824 */
/*0060*/                   IMAD.WIDE R2, R0, R5, c[0x0][0x188] ;                /* 0x0000620000027625 */
/*0070*/                   LDG.E.64.SYS R2, [R2] ;                              /* 0x0000000002027381 */
/*0080*/                   IADD3 R4, P1, R0, c[0x0][0x180], RZ ;                /* 0x0000600000047a10 */
/*0090*/                   LEA.HI.X.SX32 R5, R0, c[0x0]

/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;         /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                   /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_TID.X ;                              /* 0x0000000000007919 */
/*0030*/                   MOV R5, 0x4 ;                                   /* 0x0000000400057802 */
/*0040*/                   S2R R3, SR_CTAID.X ;                            /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R3, 0x200, R0 ;                        /* 0x0000020003007824 */
/*0060*/                   IMAD.WIDE R2, R0, R5, c[0x0][0x178] ;           /* 0x00005e0000027625 */
/*0070*/                   IMAD.WIDE R4, R0, R5, c[0x0][0x180] ;           /* 0x0000600000047625 */
/*0080*/                   LDG.E.SYS R2, [R2] ;                            /* 0x0000000002027381 */
/*0090*/                   LDG.E.SYS R5, [R4] ;                            /* 0x0000000004057381 */


/*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;         /* 0x00000a00ff017624 */
/*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                   /* 0x000000fffffff389 */
/*0020*/                   S2R R0, SR_TID.X ;                              /* 0x0000000000007919 */
/*0030*/                   MOV R5, 0x8 ;                                   /* 0x0000000800057802 */
/*0040*/                   S2R R3, SR_CTAID.X ;                            /* 0x0000000000037919 */
/*0050*/                   IMAD R0, R3, 0x200, R0 ;                        /* 0x0000020003007824 */
/*0060*/                   IMAD.WIDE R2, R0, R5, c[0x0][0x188] ;           /* 0x0000620000027625 */
/*0070*/                   LDG.E.64.SYS R2, [R2] ;                         /* 0x0000000002027381 */
/*0080*/                   IADD3 R4, P1, R0, c[0x0][0x180], RZ ;           /* 0x0000600000047a10 */
/*0090*/                   LEA.HI.X.SX32 R5, R0, c[0x0][0x184], 0x1, P1 ;  /* 0x0000610000057a11 */
